# 第6章 固有表現認識


## 6.2 データセット・前処理・評価指標

In [22]:
import sys
print(sys.version)


3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


#### 準備

In [1]:
!pip install 'datasets==2.18.0' 'transformers[ja,torch]==4.41.2'  spacy-alignments seqeval

### 6.2.1 データセット

#### データセットのダウンロード

In [2]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1461: FutureWarning: The repository for llm-book/ner-wikipedia-dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/ner-wikipedia-dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major rele

In [3]:
# データセットの形式と事例数を確認する
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [4]:
from pprint import pprint

# 訓練セットの最初の二つの事例を表示する
pprint(list(dataset["train"])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


#### データセットの分析

In [5]:
from collections import Counter
import pandas as pd
from datasets import Dataset

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:
    """固有表現タイプの出現回数をカウント"""
    # 各事例から固有表現タイプを抽出したlistを作成する
    entities = [
        e["type"] for data in dataset for e in data["entities"]
    ]

    # ラベルの出現回数が多い順に並び替える
    label_counts = dict(Counter(entities).most_common())
    return label_counts

label_counts_dict = {}
for split in dataset: # 各分割セットを処理する
    label_counts_dict[split] = count_label_occurrences(dataset[split])
# DataFrame形式で表示する
df = pd.DataFrame(label_counts_dict)
df.loc["合計"] = df.sum()
display(df)

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


#### スパンの重なる固有表現の存在を判定

In [6]:
def has_overlap(spans: list[tuple[int, int]]) -> int:
    """スパンの重なる固有表現の存在を判定"""
     # スパンを開始位置で昇順に並び替える
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        # 前のスパンの終了位置が現在のスパンの開始位置より大きい場合、
        # 重なっているとする
        if sorted_spans[i - 1][1] > sorted_spans[i][0]:
            return 1
    return 0

# 各分割セットでスパンの重なる固有表現がある事例数を数える
overlap_count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        if data["entities"]: # 固有表現の存在しない事例はスキップする
            # スパンのみのlistを作成する
            spans = [e["span"] for e in data["entities"]]
            overlap_count += has_overlap(spans)
    print(f"{split}におけるスパンが重複する事例数: {overlap_count}")

trainにおけるスパンが重複する事例数: 0
validationにおけるスパンが重複する事例数: 0
testにおけるスパンが重複する事例数: 0


### 6.2.2 前処理

#### テキストの正規化

In [7]:
from unicodedata import normalize

# テキストに対してUnicode正規化を行う
text = "ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123
正規化後: ABCABCabcabcアイウアイウ123123


In [8]:
text = "㈱、3㌕、10℃"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ㈱、3㌕、10℃
正規化後: (株)、3キログラム、10°C


In [9]:
from unicodedata import is_normalized

count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        # テキストが正規化されていない事例をカウントする
        if not is_normalized("NFKC", data["text"]):
            count += 1
print(f"正規化されていない事例数: {count}")

正規化されていない事例数: 0


#### テキストのトークナイゼーション

In [10]:
from transformers import AutoTokenizer

# トークナイザを読み込む
model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# トークナイゼーションを行う
subwords = "/".join(tokenizer.tokenize(dataset["train"][0]["text"]))
characters = "/".join(dataset["train"][0]["text"])
print(f"サブワード単位: {subwords}")
print(f"文字単位: {characters}")

サブワード単位: さくら/学院/、/C/##ia/##o/Sm/##ile/##s/の/メンバー/。
文字単位: さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。


#### 文字列とトークン列のアライメント

In [11]:
text = "さくら学院"

In [12]:
from spacy_alignments.tokenizations import get_alignments

# 文字のlistを獲得する
characters = list(text)
# テキストを特殊トークンを含めたトークンのlistに変換する
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
# 文字のlistとトークンのlistのアライメントをとる
char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(f"文字のlist: {characters}")
print(f"トークンのlist: {tokens}")
print(f"文字に対するトークンの位置: {char_to_token_indices}")
print(f"トークンに対する文字の位置: {token_to_char_indices}")

文字のlist: ['さ', 'く', 'ら', '学', '院']
トークンのlist: ['[CLS]', 'さくら', '学院', '[SEP]']
文字に対するトークンの位置: [[1], [1], [1], [2], [2]]
トークンに対する文字の位置: [[], [0, 1, 2], [3, 4], []]


#### 系列ラベリングのためのラベル作成

In [13]:
text = "大谷翔平は岩手県水沢市出身"
entities = [
    {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
    {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"},
]

In [14]:
from transformers import PreTrainedTokenizer

def output_tokens_and_labels(
    text: str,
    entities: list[dict[str, list[int] | str]],
    tokenizer: PreTrainedTokenizer,
) -> tuple[list[str], list[str]]:
    """トークンのlistとラベルのlistを出力"""
    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)

    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(tokens)
    for entity in entities: # 各固有表現で処理する
        entity_span, entity_type = entity["span"], entity["type"]
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1] - 1][0]
        # 固有表現の開始トークンの位置に"B-"のラベルを設定する
        labels[start] = f"B-{entity_type}"
        # 固有表現の開始トークン以外の位置に"I-"のラベルを設定する
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
    # 特殊トークンの位置にはラベルを設定しない
    labels[0] = "-"
    labels[-1] = "-"
    return tokens, labels

# トークンとラベルのlistを出力する
tokens, labels = output_tokens_and_labels(text, entities, tokenizer)
# DataFrameの形式で表示する
df = pd.DataFrame({"トークン列": tokens, "ラベル列": labels})
df.index.name = "位置"
display(df.T)

位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,O,B-地名,I-地名,I-地名,I-地名,O,-


### 6.2.3 評価指標

#### seqevalライブラリを用いた評価スコアの算出

In [15]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"のラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels

def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [16]:
results = [
    {
        "text": "大谷翔平は岩手県水沢市出身",
        "entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"},
        ],
        "pred_entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県", "span": [5, 8], "type": "地名"},
            {"name": "水沢市", "span": [8, 11], "type": "施設名"},
        ],
    }
]

# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を取得して表示する
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          人名       1.00      1.00      1.00         1
          地名       0.00      0.00      0.00         1
         施設名       0.00      0.00      0.00         0

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(
    true_labels: list[list[str]], pred_labels: list[list[str]], average: str
) -> dict[str, float]:
    """適合率、再現率、F値を算出"""
    scores = {
        "precision": precision_score(true_labels, pred_labels, average=average),
        "recall": recall_score(true_labels, pred_labels, average=average),
        "f1-score": f1_score(true_labels, pred_labels, average=average),
    }
    return scores

# 適合率、再現率、F値のマイクロ平均を算出する
print(compute_scores(true_labels, pred_labels, "micro"))

{'precision': np.float64(0.3333333333333333), 'recall': np.float64(0.5), 'f1-score': np.float64(0.4)}


## 6.3 固有表現認識モデルの実装

### 6.3.1 BERTのファインチューニング

#### ラベルとIDを対応づけるdictの作成

In [18]:
import torch

def create_label2id(
    entities_list: list[list[dict[str, str | int]]]
) -> dict[str, int]:
    """ラベルとIDを紐付けるdictを作成"""
    # "O"のIDには0を割り当てる
    label2id = {"O": 0}
    # 固有表現タイプのsetを獲得して並び替える
    entity_types = set(
        [e["type"] for entities in entities_list for e in entities]
    )
    entity_types = sorted(entity_types)
    for i, entity_type in enumerate(entity_types):
        # "B-"のIDには奇数番号を割り当てる
        label2id[f"B-{entity_type}"] = i * 2 + 1
        # "I-"のIDには偶数番号を割り当てる
        label2id[f"I-{entity_type}"] = i * 2 + 2
    return label2id

# ラベルとIDを紐付けるdictを作成する
label2id = create_label2id(dataset["train"]["entities"])
id2label = {v: k for k, v in label2id.items()}
pprint(id2label)

{0: 'O',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


#### データの前処理

In [19]:
from transformers.tokenization_utils_base import BatchEncoding

def preprocess_data(
    data: dict[str, Any],
    tokenizer: PreTrainedTokenizer,
    label2id: dict[int, str],
) -> BatchEncoding:
    """データの前処理"""
    # テキストのトークナイゼーションを行う
    inputs = tokenizer(
        data["text"],
        return_tensors="pt",
        return_special_tokens_mask=True,
    )
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}

    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(data["text"])
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
    char_to_token_indices, _ = get_alignments(characters, tokens)

    # "O"のIDのlistを作成する
    labels = torch.zeros_like(inputs["input_ids"])
    for entity in data["entities"]: # 各固有表現を処理する
        start_token_indices = char_to_token_indices[entity["span"][0]]
        end_token_indices = char_to_token_indices[
            entity["span"][1] - 1
        ]
        # 文字に対応するトークンが存在しなければスキップする
        if (
            len(start_token_indices) == 0
            or len(end_token_indices) == 0
        ):
            continue
        start, end = start_token_indices[0], end_token_indices[0]
        entity_type = entity["type"]
        # 固有表現の開始トークンの位置に"B-"のIDを設定する
        labels[start] = label2id[f"B-{entity_type}"]
        # 固有表現の開始トークン以外の位置に"I-"のIDを設定する
        if start != end:
            labels[start + 1 : end + 1] = label2id[f"I-{entity_type}"]
    # 特殊トークンの位置のIDは-100とする
    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs["labels"] = labels
    return inputs

# 訓練セットに対して前処理を行う
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["train"].column_names,
)
# 検証セットに対して前処理を行う
validation_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["validation"].column_names,
)

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'label2id': {'O': 0, 'B-その他の組織名': 1, 'I-その他の組織名': 2, 'B-イベント名': 3, 'I-イベント名': 4

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

#### モデルの準備

In [20]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
)

# モデルを読み込む
model = AutoModelForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)
# パラメータをメモリ上に隣接した形で配置
# これを実行しない場合、モデルの保存でエラーになることがある
for param in model.parameters():
    param.data = param.data.contiguous()
# collate関数にDataCollatorForTokenClassificationを用いる
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### モデルのファインチューニング

In [21]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="output_bert_ner", # 結果の保存フォルダ
    per_device_train_batch_size=32, # 訓練時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=1e-4, # 学習率
    lr_scheduler_type="linear", # 学習率スケジューラ
    warmup_ratio=0.1, # 学習率のウォームアップ
    num_train_epochs=5, # 訓練エポック数
    evaluation_strategy="epoch", # 評価タイミング
    save_strategy="epoch", # チェックポイントの保存タイミング
    logging_strategy="epoch", # ロギングのタイミング
    fp16=True, # 自動混合精度演算の有効化
    report_to="none",  # 外部ツールへのログを無効化
)

# Trainerを初期化する
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
from google.colab import drive

# Googleドライブをマウントする
drive.mount("drive")

Mounted at drive


In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!mkdir -p drive/MyDrive/llm-book
!cp -r output_bert_ner drive/MyDrive/llm-book

### 6.3.2 固有表現の予測・抽出

#### 固有表現ラベルの予測

In [ ]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
    """ミニバッチのデータを事例単位のlistに変換"""
    dict_list = []
    # dictのキーのlistを作成する
    keys = list(list_dict.keys())
    for idx in range(len(list_dict[keys[0]])): # 各事例で処理する
        # dictの各キーからデータを取り出してlistに追加する
        dict_list.append({key: list_dict[key][idx] for key in keys})
    return dict_list

# ミニバッチのデータを事例単位のlistに変換する
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]],
}
dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"入力: {list_dict}")
print(f"出力: {dict_list}")

入力: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
出力: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, model: PreTrainedModel
) -> list[dict[str, Any]]:
    """予測スコアに基づき固有表現ラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader): # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # 予測スコアを取得する
        logits = model(**inputs).logits
        # 最もスコアの高いIDを取得する
        batch["pred_label_ids"] = logits.argmax(-1)
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ミニバッチの作成にDataLoaderを用いる
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"])

100%|██████████| 17/17 [00:01<00:00, 13.14it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 13, 14, 14, 14, 13, 0, 0, 0, 0, 0, 15, 15, 16, 15, 13, 14, 14, 14, 13, 13, 13, 13, 14, 14, 0, 0, 0, 0, 13, 14, 14, 14, 0, 0, 0, 14, 14, 14, 0, 0, 0, 15, 0, 15, 16, 16, 15, 13, 14, 14, 14, 14, 0, 0, 15, 15, 15, 15, 16, 0, 13, 13, 14]


#### 固有表現の抽出

In [ ]:
from seqeval.metrics.sequence_labeling import get_entities

def extract_entities(
    predictions: list[dict[str, Any]],
    dataset: list[dict[str, Any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str],
) -> list[dict[str, Any]]:
    """固有表現を抽出"""
    results = []
    for prediction, data in zip(predictions, dataset):
        # 文字のlistを取得する
        characters = list(data["text"])

        # 特殊トークンを除いたトークンのlistと予測ラベルのlistを取得する
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(
            prediction["input_ids"]
        )
        for token, label_id in zip(
            all_tokens, prediction["pred_label_ids"]
        ):
            # 特殊トークン以外をlistに追加する
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])

        # 文字のlistとトークンのlistのアライメントをとる
        _, token_to_char_indices = get_alignments(characters, tokens)

        # 予測ラベルのlistから固有表現タイプと、
        # トークン単位の開始位置と終了位置を取得して、
        # それらを正解データと同じ形式に変換する
        pred_entities = []
        for entity in get_entities(pred_labels):
            entity_type, token_start, token_end = entity
            # 文字単位の開始位置を取得する
            char_start = token_to_char_indices[token_start][0]
            # 文字単位の終了位置を取得する
            char_end = token_to_char_indices[token_end][-1] + 1
            pred_entity = {
                "name": "".join(characters[char_start:char_end]),
                "span": [char_start, char_end],
                "type": entity_type,
            }
            pred_entities.append(pred_entity)
        data["pred_entities"] = pred_entities
        results.append(data)
    return results

# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)
pprint(results[0])

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


### 6.3.3 検証セットを使ったモデルの選択

In [ ]:
from glob import glob

best_score = 0
# 各チェックポイントで処理する
for checkpoint in sorted(glob("output_bert_ner/checkpoint-*")):
    # モデルを読み込む
    model = AutoModelForTokenClassification.from_pretrained(
        checkpoint
    )
    model.to("cuda:0")  # モデルをGPUに移動
    # 固有表現ラベルを予測する
    predictions = run_prediction(validation_dataloader, model)
    # 固有表現を抽出する
    results = extract_entities(
        predictions, dataset["validation"], tokenizer, id2label
    )
    # 正解データと予測データのラベルのlistを作成する
    true_labels, pred_labels = convert_results_to_labels(results)
    # 評価スコアを算出する
    scores = compute_scores(true_labels, pred_labels, "micro")
    if best_score < scores["f1-score"]:
        best_score = scores["f1-score"]
        best_model = model

100%|██████████| 17/17 [00:03<00:00,  5.62it/s]


### 6.3.4 性能評価

In [ ]:
# モデルを読み込む
model_name = "llm-book/bert-base-japanese-v3-ner-wikipedia-dataset"
best_model = AutoModelForTokenClassification.from_pretrained(
    model_name
)
best_model = best_model.to("cuda:0")

In [ ]:
# テストセットに対して前処理を行う
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["test"].column_names,
)
# ミニバッチの作成にDataLoaderを用いる
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

100%|██████████| 17/17 [00:03<00:00,  5.07it/s]


              precision    recall  f1-score   support

     その他の組織名       0.86      0.83      0.84       100
       イベント名       0.82      0.92      0.87        93
          人名       0.96      0.96      0.96       287
          地名       0.85      0.86      0.86       204
      政治的組織名       0.75      0.89      0.81       106
         施設名       0.85      0.85      0.85       137
         法人名       0.89      0.88      0.89       248
         製品名       0.76      0.81      0.79       158

   micro avg       0.86      0.88      0.87      1333
   macro avg       0.84      0.88      0.86      1333
weighted avg       0.86      0.88      0.87      1333



### 6.3.5 エラー分析

In [ ]:
def find_error_results(
    results: list[dict[str, Any]],
) -> list[dict[str, Any]]:
    """エラー事例を発見"""
    error_results = []
    for idx, result in enumerate(results): # 各事例を処理する
        result["idx"] = idx
        # 正解データと予測データが異なるならばlistに加える
        if result["entities"] != result["pred_entities"]:
            error_results.append(result)
    return error_results

def output_text_with_label(result: dict[str, Any], entity_column: str) -> str:
    """固有表現ラベル付きテキストを出力"""
    text_with_label = ""
    entity_count = 0
    for i, char in enumerate(result["text"]): # 各文字を処理する
        # 出力に加えていない固有表現の有無を判定する
        if entity_count < len(result[entity_column]):
            entity = result[entity_column][entity_count]
            # 固有表現の先頭の処理を行う
            if i == entity["span"][0]:
                entity_type = entity["type"]
                text_with_label += f" [({entity_type}) "
            text_with_label += char
            # 固有表現の末尾の処理を行う
            if i == entity["span"][1] - 1:
                text_with_label += "] "
                entity_count += 1
        else:
            text_with_label += char
    return text_with_label

# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:3]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")
    print()

事例18の正解:  [(法人名) 常盤木学園] 時代の同級生に [(その他の組織名) なでしこジャパン] の [(人名) 熊谷紗希] がいる。
事例18の予測:  [(施設名) 常盤木学園] 時代の同級生に [(その他の組織名) なでしこジャパン] の [(人名) 熊谷紗希] がいる。

事例19の正解: テレビで狼男映画の「 [(製品名) 倫敦の人狼] 」を見た [(人名) フィル・エヴァリー] は「ロンドンの狼男というタイトルで踊り騒げる曲を書いてみないか」と [(法人名) ジヴォン] に持ちかけた。
事例19の予測: テレビで狼男映画の「 [(製品名) 倫敦の人狼] 」を見た [(人名) フィル・エヴァリー] は「 [(製品名) ロンドン] の [(製品名) 狼男] というタイトルで踊り騒げる曲を書いてみないか」と [(人名) ジヴォン] に持ちかけた。

事例27の正解:  [(政治的組織名) 李承晩政権] 期から [(政治的組織名) 朴正煕政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。
事例27の予測:  [(政治的組織名) 李]  [(人名) 承晩]  [(政治的組織名) 政権] 期から [(政治的組織名) 朴]  [(人名) 正煕]  [(政治的組織名) 政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。



### 6.3.6 ラベル間の遷移可能性を考慮した予測

#### 遷移スコアを定義

In [ ]:
def create_transitions(
    label2id: dict[str, int]
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """遷移スコアを定義"""
    # "B-"のラベルIDのlist
    b_ids = [v for k, v in label2id.items() if k[0] == "B"]
    # I-のラベルIDのlist
    i_ids = [v for k, v in label2id.items() if k[0] == "I"]
    o_id = label2id["O"]  # OのラベルID

    # 開始遷移スコアを定義する
    # すべてのスコアを-100で初期化する
    start_transitions = torch.full([len(label2id)], -100.0)
    # "B-"のラベルへ遷移可能として0を代入する
    start_transitions[b_ids] = 0
    # "O"のラベルへ遷移可能として0を代入する
    start_transitions[o_id] = 0

    # ラベル間の遷移スコアを定義する
    # すべてのスコアを-100で初期化する
    transitions = torch.full([len(label2id), len(label2id)], -100.0)
    # すべてのラベルから"B-"へ遷移可能として0を代入する
    transitions[:, b_ids] = 0
    # すべてのラベルから"O"へ遷移可能として0を代入する
    transitions[:, o_id] = 0
    # "B-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[b_ids, i_ids] = 0
    # "I-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[i_ids, i_ids] = 0

    # 終了遷移スコアを定義する
    # すべてのラベルから遷移可能としてすべてのスコアを0とする
    end_transitions = torch.zeros(len(label2id))
    return start_transitions, transitions, end_transitions

# 遷移スコアを定義する
start_transitions, transitions, end_transitions = create_transitions(
    label2id
)

#### ビタビアルゴリズムを用いたラベル列の予測

In [ ]:
def decode_with_viterbi(
    emissions: torch.Tensor,  # ラベルの予測スコア
    mask: torch.Tensor,  # マスク
    start_transitions: torch.Tensor,  # 開始遷移スコア
    transitions: torch.Tensor,  # ラベル間の遷移スコア
    end_transitions: torch.Tensor,  # 終了遷移スコア
) -> torch.Tensor:
    """ビタビアルゴリズムを用いて最適なラベル列を探索"""
    # バッチサイズと系列長を取得する
    batch_size, seq_length = mask.shape
    # 予測スコアとマスクに関して、0次元目と1次元目を入れ替える
    emissions = emissions.transpose(1, 0)
    mask = mask.transpose(1, 0)

    histories = []  # 最適なラベル系列を保存するための履歴のlist
    # 開始遷移スコアと予測スコアを加算して、累積スコアの初期値とする
    score = start_transitions + emissions[0]
    for i in range(1, seq_length):
        # 累積スコアを3次元に変換する
        broadcast_score = score.unsqueeze(2)
        # 現在の予測スコアを3次元に変換する
        broadcast_emission = emissions[i].unsqueeze(1)
        # 累積スコアと遷移スコアと現在の予測スコアを加算して、
        # 現在の累積スコアを取得する
        next_score = (
            broadcast_score + transitions + broadcast_emission
        )
        # 現在の累積スコアの各ラベルの最大値とそのインデックスを取得する
        next_score, indices = next_score.max(dim=1)
        # マスクしない要素の場合、累積スコアを更新する
        score = torch.where(mask[i].unsqueeze(1), next_score, score)
        # スコアの高いインデックスを履歴のlistに追加する
        histories.append(indices)
    # 終了遷移スコアを加算して合計スコアとする
    score += end_transitions

    # 各事例で最適なラベル列を取得する
    best_labels_list = []
    for i in range(batch_size):
        # 合計スコアの中で最大のスコアとなるラベルを取得する
        _, best_last_label = score[i].max(dim=0)
        best_labels = [best_last_label.item()]
        # 最後のラベルの遷移を逆方向に探索し、最適なラベル列を取得する
        for history in reversed(histories):
            best_last_label = history[i][best_labels[-1]]
            best_labels.append(best_last_label.item())
        # 順序を反転する
        best_labels.reverse()
        best_labels_list.append(best_labels)
    return torch.LongTensor(best_labels_list)

In [ ]:
def run_prediction_viterbi(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """ビダビアルゴリズムを用いてラベルを予測"""
    # 遷移スコアを取得する
    start_transitions, transitions, end_transitions = (
        create_transitions(model.config.label2id)
    )

    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ビタビアルゴリズムを用いてラベルを予測する
predictions = run_prediction_viterbi(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:03<00:00,  4.45it/s]


              precision    recall  f1-score   support

     その他の組織名       0.86      0.83      0.85       100
       イベント名       0.84      0.94      0.89        93
          人名       0.96      0.96      0.96       287
          地名       0.87      0.87      0.87       204
      政治的組織名       0.79      0.91      0.85       106
         施設名       0.88      0.86      0.87       137
         法人名       0.90      0.88      0.89       248
         製品名       0.79      0.81      0.80       158

   micro avg       0.88      0.89      0.88      1333
   macro avg       0.86      0.88      0.87      1333
weighted avg       0.88      0.89      0.88      1333



In [ ]:
idx = 27
result = results[idx]
true_text = output_text_with_label(result, "entities")
pred_text = output_text_with_label(result, "pred_entities")
print(f"事例{idx}の正解: {true_text}")
print(f"事例{idx}の予測: {pred_text}")

事例27の正解:  [(政治的組織名) 李承晩政権] 期から [(政治的組織名) 朴正煕政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。
事例27の予測:  [(政治的組織名) 李承晩政権] 期から [(政治的組織名) 朴正煕政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。


### 6.3.7 CRFによるラベル間の遷移可能性の学習

#### BERT-CRFモデルの実装

In [ ]:
!pip install pytorch-crf

In [ ]:
from torchcrf import CRF
from transformers import BertForTokenClassification, PretrainedConfig
from transformers.modeling_outputs import TokenClassifierOutput

class BertWithCrfForTokenClassification(BertForTokenClassification):
    """BertForTokenClassificationにCRF層を加えたクラス"""

    def __init__(self, config: PretrainedConfig):
        """クラスの初期化"""
        super().__init__(config)
        # CRF層を定義する
        self.crf = CRF(len(config.label2id), batch_first=True)

    def _init_weights(self, module: torch.nn.Module) -> None:
        """定義した遷移スコアでパラメータを初期化"""
        super()._init_weights(module)
        if isinstance(module, CRF):
            st, t, et = create_transitions(self.config.label2id)
            module.start_transitions.data = st
            module.transitions.data = t
            module.end_transitions.data = et

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor | None = None,
        token_type_ids: torch.Tensor | None = None,
        labels: torch.Tensor | None = None,
    ) -> TokenClassifierOutput:
        """モデルの前向き計算を定義"""
        # BertForTokenClassificationのforwardメソッドを適用して
        # 予測スコアを取得する
        output = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        if labels is not None:
            logits = output.logits
            mask = labels != -100
            labels *= mask
            # CRFによる損失を計算する
            output["loss"] = -self.crf(
                logits[:, 1:, :],
                labels[:, 1:],
                mask=mask[:, 1:],
                reduction="mean",
            )
        return output

# BertForTokenClassificationにCRF層を加えたクラスを定義する
model_crf = BertWithCrfForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)

Some weights of BertWithCrfForTokenClassification were not initialized from the model checkpoint at llm-book/bert-base-japanese-v3-ner-wikipedia-dataset and are newly initialized: ['crf.transitions', 'crf.start_transitions', 'crf.end_transitions']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 乱数のシード値を再設定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="output_bert_crf_ner", # 結果の保存フォルダ
    per_device_train_batch_size=32, # 訓練時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=1e-4, # 学習率
    lr_scheduler_type="linear", # 学習率スケジューラ
    warmup_ratio=0.1, # 学習率のウォームアップ
    num_train_epochs=5, # 訓練エポック数
    evaluation_strategy="epoch", # 評価タイミング
    save_strategy="epoch", # チェックポイントの保存タイミング
    logging_strategy="epoch", # ロギングのタイミング
    fp16=True, # 自動混合精度演算の有効化
)

# Trainerを初期化する
trainer = Trainer(
    model=model_crf,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.332500,3.026173
2,0.393900,2.945430
3,0.303100,2.843552
4,0.170500,3.344096
5,0.144500,3.401141


TrainOutput(global_step=670, training_loss=0.2689061919255043, metrics={'train_runtime': 262.1517, 'train_samples_per_second': 81.518, 'train_steps_per_second': 2.556, 'total_flos': 1054777482669504.0, 'train_loss': 0.2689061919255043, 'epoch': 5.0})

In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!cp -r output_bert_crf_ner drive/MyDrive/llm-book

In [ ]:
def run_prediction_crf(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """BERT-CRFモデルを用いてラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"] == 0).cpu()[:, 1:]
        # 訓練した遷移スコアを取得する
        start_transitions = model.crf.start_transitions.cpu()
        transitions = model.crf.transitions.cpu()
        end_transitions = model.crf.end_transitions.cpu()
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

In [ ]:
best_score = 0
# 各チェックポイントで処理する
for checkpoint in sorted(glob("output_bert_crf_ner/checkpoint-*")):
    # モデルを読み込む
    model_crf = BertWithCrfForTokenClassification.from_pretrained(
        checkpoint
    )
    model_crf = model_crf.to("cuda:0")  # モデルをGPUに移動
    # 固有表現ラベルを予測する
    predictions = run_prediction_crf(validation_dataloader, model_crf)
    # 固有表現を抽出する
    results = extract_entities(
        predictions, dataset["validation"], tokenizer, id2label
    )
    # 正解データと予測データのラベルのlistを作成する
    true_labels, pred_labels = convert_results_to_labels(results)
    # 評価スコアを算出する
    scores = compute_scores(true_labels, pred_labels, "micro")
    if best_score < scores["f1-score"]:
        best_score = scores["f1-score"]
        best_model_crf = model_crf

100%|██████████| 17/17 [00:03<00:00,  5.00it/s]


In [ ]:
# モデルを読み込む
model_name = "llm-book/bert-base-japanese-v3-crf-ner-wikipedia-dataset"
best_model_crf = BertWithCrfForTokenClassification.from_pretrained(
    model_name
)
best_model_crf = best_model_crf.to("cuda:0")

In [ ]:
# 固有表現ラベルを予測する
predictions = run_prediction_crf(test_dataloader, best_model_crf)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:03<00:00,  4.34it/s]


              precision    recall  f1-score   support

     その他の組織名       0.85      0.79      0.82       100
       イベント名       0.92      0.94      0.93        93
          人名       0.96      0.96      0.96       287
          地名       0.90      0.88      0.89       204
      政治的組織名       0.84      0.89      0.86       106
         施設名       0.91      0.91      0.91       137
         法人名       0.88      0.89      0.88       248
         製品名       0.84      0.84      0.84       158

   micro avg       0.89      0.90      0.90      1333
   macro avg       0.89      0.89      0.89      1333
weighted avg       0.89      0.90      0.90      1333



## 6.4 アノテーションツールを用いたデータセット構築

### 6.4.3 構築したデータセットでの性能評価

In [ ]:
# データセットを読み込む
dataset_wikinews = load_dataset("llm-book/ner-wikinews-dataset")
print(dataset_wikinews)

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ner-wikinews-dataset downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___ner-wikinews-dataset/new-wikinews-dataset/1.0.0/5512ac5f96221077ce49b424f8b5c88c7e7afb5a92e33c2f4c9049e9a170a894. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 178
    })
})


In [ ]:
# テストセットの前処理を行う
test_dataset_wikinews = dataset_wikinews["test"].map(
    preprocess_data,
    fn_kwargs={"tokenizer": tokenizer, "label2id": label2id},
    remove_columns=dataset_wikinews["test"].column_names
)
# ミニバッチの作成にDataLoaderを用いる
test_dataloader_wikinews = DataLoader(
    test_dataset_wikinews,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

In [ ]:
# 固有表現ラベルを予測する
predictions = run_prediction_crf(
    test_dataloader_wikinews, best_model_crf
)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset_wikinews["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を表示する
print(classification_report(true_labels, pred_labels))

100%|██████████| 6/6 [00:01<00:00,  5.61it/s]


              precision    recall  f1-score   support

     その他の組織名       0.89      0.67      0.76        12
       イベント名       0.79      0.60      0.68        25
          人名       0.98      0.98      0.98        62
          地名       0.82      0.86      0.84       143
      政治的組織名       0.83      0.90      0.86        49
         施設名       0.77      0.85      0.81        27
         法人名       0.88      0.80      0.84        56
         製品名       0.79      0.77      0.78        39

   micro avg       0.85      0.85      0.85       413
   macro avg       0.84      0.80      0.82       413
weighted avg       0.85      0.85      0.84       413



In [ ]:
# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:3]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")
    print()

事例3の正解:  [(人名) 岸田] 夫人は、 [(人名) ジル] 夫人に2023年5月に [(地名) 広島] で開かれる [(イベント名) G7広島サミット] で、訪日してくれることを楽しみにしていると伝えた。
事例3の予測:  [(人名) 岸田] 夫人は、 [(人名) ジル] 夫人に2023年5月に [(地名) 広島] で開かれる [(イベント名) G7広島サミット] で、訪 [(地名) 日] してくれることを楽しみにしていると伝えた。

事例9の正解: 新庁舎は [(施設名) 旧京都府警本部] を改修した建物と新たに建設した建物となる。
事例9の予測: 新庁舎は [(政治的組織名) 旧京都府警本部] を改修した建物と新たに建設した建物となる。

事例10の正解: 新庁舎は [(地名) 京都府] が整備を行い、 [(政治的組織名) 文化庁] に貸し出されることになっている。
事例10の予測: 新庁舎は [(政治的組織名) 京都府] が整備を行い、 [(政治的組織名) 文化庁] に貸し出されることになっている。

